In [1]:
from cassandra.cluster import Cluster
from tqdm import tqdm
import datetime
import gzip
import json
import sys
import re

#### Parser sobre o arquivo de entrada ``amazon-meta.txt.gz`` para gerar um arquivo que tem os inserts

In [4]:
def fixAps(string):
    return string.replace("'", "''")

In [2]:
def get_line_number(file_path):
    sys.stderr.write("Counting line number of {}".format(file_path))
    
    with gzip.open(file_path, 'rb') as file:
        for lines, l in enumerate(file):
            pass
        
    return lines

In [3]:
def parser(filename, total):
    IGNORE_FIELDS = ['Total items', 'reviews']
    f = gzip.open(filename, 'r')
    entry = {}
    categories = []
    reviews = []
    similar_items = []
    
    for line in tqdm(f, total=total):
        line = line.decode("utf-8").strip()
        colonPos = line.find(':')
        
        if line.startswith("Id"):
            if reviews:
                entry["reviews"] = reviews
            if categories:
                entry["categories"] = categories
            
            yield entry
            entry = {}
            categories = []
            reviews = []
            rest = line[colonPos+2:]
            entry["_id"] = int(rest.strip())
        
        elif line.startswith("similar"):
            similar_items = line.split()[2:]
            entry['similar_items'] = similar_items
            
        elif line.find("cutomer:") != -1:
            review_info = line.split()
            reviews.append({'data': review_info[0],'customer_id': review_info[2],'rating': int(review_info[4]),'votes': int(review_info[6]),'helpful': int(review_info[8])})
            
        elif line.startswith("|"):
            categories.append(line)
            
        elif colonPos != -1:
            eName = line[:colonPos]
            rest = line[colonPos+2:]
            
            if not eName in IGNORE_FIELDS:
                entry[eName] = rest.strip()
                if(eName == 'salesrank'):
                    entry[eName] = int(entry[eName])
                
    if reviews:
        entry["reviews"] = reviews
        
    if categories:
        entry["categories"] = categories
        
    yield entry

In [37]:
path_file = 'amazon-meta.txt.gz'

In [7]:
line_num = get_line_number(path_file)

Counting line number of amazon-meta.txt.gz

In [40]:
with open('amazon-meta.json', 'w') as f:
    for e in parser(path_file, total=line_num):
        if e:
            json.dump(e, f)
            f.write('\n')






  0%|          | 0/15010573 [00:00<?, ?it/s]




  0%|          | 10293/15010573 [00:00<02:25, 102880.31it/s]




  0%|          | 21615/15010573 [00:00<02:21, 105724.67it/s]




  0%|          | 32581/15010573 [00:00<02:20, 106867.27it/s]




  0%|          | 45309/15010573 [00:00<02:13, 112200.08it/s]




  0%|          | 57460/15010573 [00:00<02:10, 114836.59it/s]




  0%|          | 69477/15010573 [00:00<02:08, 116385.78it/s]




  1%|          | 81399/15010573 [00:00<02:07, 117159.95it/s]




  1%|          | 94038/15010573 [00:00<02:04, 119725.19it/s]




  1%|          | 106635/15010573 [00:00<02:02, 121495.70it/s]




  1%|          | 118406/15010573 [00:01<02:04, 120054.84it/s]




  1%|          | 130775/15010573 [00:01<02:02, 121080.73it/s]




  1%|          | 142702/15010573 [00:01<02:06, 117529.56it/s]




  1%|          | 154719/15010573 [00:01<02:05, 118236.61it/s]




  1%|          | 167426/15010573 [00:01<02:02, 120731.90it/s]




  1%|          | 179462/150105

 15%|█▌        | 2269899/15010573 [00:21<01:58, 107767.62it/s]




 15%|█▌        | 2281024/15010573 [00:21<01:57, 107939.46it/s]




 15%|█▌        | 2291857/15010573 [00:21<02:00, 105782.36it/s]




 15%|█▌        | 2302472/15010573 [00:21<02:06, 100357.07it/s]




 15%|█▌        | 2312592/15010573 [00:21<02:14, 94100.08it/s] 




 15%|█▌        | 2323273/15010573 [00:21<02:10, 97497.70it/s]




 16%|█▌        | 2333328/15010573 [00:21<02:08, 98383.32it/s]




 16%|█▌        | 2344440/15010573 [00:21<02:04, 101815.34it/s]




 16%|█▌        | 2356322/15010573 [00:22<01:58, 106382.51it/s]




 16%|█▌        | 2367080/15010573 [00:22<02:01, 104275.36it/s]




 16%|█▌        | 2378269/15010573 [00:22<01:58, 106317.88it/s]




 16%|█▌        | 2389961/15010573 [00:22<01:55, 109290.57it/s]




 16%|█▌        | 2401947/15010573 [00:22<01:52, 112256.93it/s]




 16%|█▌        | 2414371/15010573 [00:22<01:49, 115544.72it/s]




 16%|█▌        | 2427170/15010573 [00:22<01:45, 119015.86it/s]



 32%|███▏      | 4736692/15010573 [00:42<01:55, 88838.04it/s]




 32%|███▏      | 4745843/15010573 [00:42<01:58, 86538.90it/s]




 32%|███▏      | 4755167/15010573 [00:42<01:55, 88435.39it/s]




 32%|███▏      | 4764964/15010573 [00:42<01:52, 91087.86it/s]




 32%|███▏      | 4774850/15010573 [00:42<01:49, 93280.94it/s]




 32%|███▏      | 4784675/15010573 [00:42<01:48, 94658.17it/s]




 32%|███▏      | 4796117/15010573 [00:42<01:42, 99820.92it/s]




 32%|███▏      | 4808811/15010573 [00:42<01:35, 106646.05it/s]




 32%|███▏      | 4820857/15010573 [00:42<01:32, 110422.57it/s]




 32%|███▏      | 4832087/15010573 [00:42<01:32, 109467.50it/s]




 32%|███▏      | 4844362/15010573 [00:43<01:29, 113140.10it/s]




 32%|███▏      | 4856579/15010573 [00:43<01:27, 115705.22it/s]




 32%|███▏      | 4868274/15010573 [00:43<01:27, 116069.55it/s]




 33%|███▎      | 4879953/15010573 [00:43<01:31, 110650.36it/s]




 33%|███▎      | 4891122/15010573 [00:43<01:31, 110165.76it/s]




 3

 40%|███▉      | 6001238/15010573 [00:52<01:09, 130231.68it/s]




 40%|████      | 6014336/15010573 [00:52<01:08, 130454.85it/s]




 40%|████      | 6027397/15010573 [00:52<01:10, 128279.44it/s]




 40%|████      | 6040244/15010573 [00:52<01:09, 128189.03it/s]




 40%|████      | 6053077/15010573 [00:52<01:09, 128158.05it/s]




 40%|████      | 6065903/15010573 [00:52<01:10, 127629.28it/s]




 41%|████      | 6079376/15010573 [00:52<01:08, 129679.05it/s]




 41%|████      | 6092510/15010573 [00:53<01:08, 130077.60it/s]




 41%|████      | 6105528/15010573 [00:53<01:08, 129972.87it/s]




 41%|████      | 6118532/15010573 [00:53<01:09, 128012.91it/s]




 41%|████      | 6132496/15010573 [00:53<01:07, 131290.52it/s]




 41%|████      | 6146051/15010573 [00:53<01:06, 132518.99it/s]




 41%|████      | 6159326/15010573 [00:53<01:08, 130082.42it/s]




 41%|████      | 6172815/15010573 [00:53<01:07, 131487.72it/s]




 41%|████      | 6185986/15010573 [00:53<01:08, 128569.38it/s]

 56%|█████▋    | 8458256/15010573 [01:12<00:54, 121229.82it/s]




 56%|█████▋    | 8471672/15010573 [01:13<00:52, 124647.28it/s]




 57%|█████▋    | 8484283/15010573 [01:13<00:52, 125063.39it/s]




 57%|█████▋    | 8496827/15010573 [01:13<00:52, 123264.10it/s]




 57%|█████▋    | 8509186/15010573 [01:13<00:52, 122894.34it/s]




 57%|█████▋    | 8521498/15010573 [01:13<00:53, 120745.80it/s]




 57%|█████▋    | 8533597/15010573 [01:13<00:54, 119117.74it/s]




 57%|█████▋    | 8545704/15010573 [01:13<00:54, 119669.06it/s]




 57%|█████▋    | 8558388/15010573 [01:13<00:53, 121053.77it/s]




 57%|█████▋    | 8570509/15010573 [01:13<00:54, 118188.66it/s]




 57%|█████▋    | 8582727/15010573 [01:13<00:53, 119279.35it/s]




 57%|█████▋    | 8596250/15010573 [01:14<00:51, 123647.55it/s]




 57%|█████▋    | 8608670/15010573 [01:14<00:53, 119040.56it/s]




 57%|█████▋    | 8620650/15010573 [01:14<00:53, 118424.82it/s]




 58%|█████▊    | 8632640/15010573 [01:14<00:53, 118861.39it/s]

 73%|███████▎  | 10953611/15010573 [01:33<00:32, 125598.20it/s]




 73%|███████▎  | 10966179/15010573 [01:33<00:32, 123690.92it/s]




 73%|███████▎  | 10978561/15010573 [01:33<00:32, 122833.37it/s]




 73%|███████▎  | 10990855/15010573 [01:33<00:32, 121819.81it/s]




 73%|███████▎  | 11003046/15010573 [01:33<00:33, 119794.79it/s]




 73%|███████▎  | 11015946/15010573 [01:33<00:32, 122414.76it/s]




 73%|███████▎  | 11029906/15010573 [01:33<00:32, 123394.36it/s]




 74%|███████▎  | 11042264/15010573 [01:33<00:32, 120790.28it/s]




 74%|███████▎  | 11054369/15010573 [01:34<00:33, 118171.58it/s]




 74%|███████▎  | 11066247/15010573 [01:34<00:33, 118346.48it/s]




 74%|███████▍  | 11078769/15010573 [01:34<00:32, 120327.35it/s]




 74%|███████▍  | 11092257/15010573 [01:34<00:32, 120318.23it/s]




 74%|███████▍  | 11104411/15010573 [01:34<00:32, 120681.44it/s]




 74%|███████▍  | 11116491/15010573 [01:34<00:33, 116952.95it/s]




 74%|███████▍  | 11128220/15010573 [01:34<00:33,

 89%|████████▉ | 13370160/15010573 [01:53<00:14, 116992.59it/s]




 89%|████████▉ | 13381879/15010573 [01:53<00:13, 116970.97it/s]




 89%|████████▉ | 13393590/15010573 [01:53<00:14, 112111.63it/s]




 89%|████████▉ | 13404976/15010573 [01:53<00:14, 112630.11it/s]




 89%|████████▉ | 13416277/15010573 [01:53<00:14, 112539.22it/s]




 89%|████████▉ | 13428204/15010573 [01:54<00:13, 114357.00it/s]




 90%|████████▉ | 13439666/15010573 [01:54<00:14, 111385.56it/s]




 90%|████████▉ | 13450840/15010573 [01:54<00:14, 109342.70it/s]




 90%|████████▉ | 13464454/15010573 [01:54<00:13, 110774.44it/s]




 90%|████████▉ | 13476223/15010573 [01:54<00:13, 112759.03it/s]




 90%|████████▉ | 13488741/15010573 [01:54<00:13, 116212.19it/s]




 90%|████████▉ | 13500769/15010573 [01:54<00:12, 117147.63it/s]




 90%|█████████ | 13512518/15010573 [01:54<00:12, 117121.34it/s]




 90%|█████████ | 13524763/15010573 [01:54<00:12, 118569.94it/s]




 90%|█████████ | 13537742/15010573 [01:54<00:12,

In [2]:
products = []

with open('amazon-meta.json', 'r') as f:
    for line in f:
        products.append(json.loads(line))

In [5]:
content = ""

for e in products:
    id_prod = "NULL"
    asin = "NULL"
    title = "NULL"
    group = "NULL"
    salesrank = "NULL"
    
    if('_id' in e):
        id_prod = e["_id"]
    if('ASIN' in e):
        asin =  e["ASIN"]
    if('title' in e):
        title = e["title"]
    if('group' in e):
        group = e["group"]
    if('salesrank' in e):
        salesrank = e["salesrank"]
        
    content += "INSERT INTO PRODUCT(id, asin, title, group_, salesrank) VALUES ({}, \'{}\', \'{}\', \'{}\', {});\n".format(id_prod, fixAps(asin), fixAps(title), fixAps(group), salesrank)
    
    if('similar_items' in e and len(e['similar_items']) > 0):
        for similar in e['similar_items']:
            content += "INSERT INTO PRODUCT_similar_asin(similar_asin, PRODUCT_similar_asin_id) VALUES (\'{}\', {});\n".format(similar, id_prod)
    
    if('categories' in e and len(e['categories']) > 0):
        for categories in e['categories']:
            content += "INSERT INTO PRODUCT_categorie(categorie, PRODUCT_categorie_id) VALUES (\'{}\', {});\n".format(fixAps(categories), id_prod)
            
    if('reviews' in e and len(e['reviews']) > 0):
        for reviews in e['reviews']:
            content += "INSERT INTO REVIEW(tem_PRODUCT_id, data, rating, votes, helpful, customer) VALUES ({}, \'{}\', {}, {}, {}, \'{}\');\n".format(id_prod, reviews['data'], reviews['rating'], reviews['votes'], reviews['helpful'], reviews['customer_id'])

In [6]:
with open('inserts.sql','w') as f:
    f.write(content)

#### Criar um keyspace com nome ``amazon_meta`` e com a replicação ``{'class':'SimpleStrategy', 'replication_factor' : 2}``

In [2]:
cluster = Cluster(['localhost'])

In [3]:
session = cluster.connect('amazon_meta')

##### Create table

In [4]:
drop_tables = ['DROP TABLE IF EXISTS PRODUCT;', \
              'DROP TABLE IF EXISTS PRODUCT_similar_asin;', \
              'DROP TABLE IF EXISTS PRODUCT_categorie;', \
              'DROP TABLE IF EXISTS REVIEW;']

In [5]:
create_tables = ['CREATE TABLE PRODUCT( id INT, \
asin TEXT, title TEXT, group_ TEXT, salesrank INT, PRIMARY KEY (id) );',
                'CREATE TABLE PRODUCT_similar_asin( similar_asin TEXT, \
PRODUCT_similar_asin_id INT, PRIMARY KEY (similar_asin, PRODUCT_similar_asin_id));',
                'CREATE TABLE PRODUCT_categorie( categorie TEXT, \
PRODUCT_categorie_id INT, PRIMARY KEY (categorie, PRODUCT_categorie_id));',
                'CREATE TABLE REVIEW( tem_PRODUCT_id INT, data TIMESTAMP, \
rating INT, votes INT, helpful INT, customer TEXT, \
PRIMARY KEY (customer, tem_PRODUCT_id));']

In [6]:
for drop_table in drop_tables:
    session.execute(drop_table)

for create_table in create_tables:
    session.execute(create_table)

##### Inserts

In [7]:
with open('inserts.sql','r') as f:
    inserts = f.readlines()

In [8]:
for i in tqdm(range(len(inserts))):
    session.execute(inserts[i][:-1])

100%|██████████| 12463123/12463123 [1:13:26<00:00, 2828.62it/s]


##### Queries

(a) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação

In [ ]:
id_product = 1

In [23]:
sql = 'SELECT helpful, rating, tem_PRODUCT_id, customer, data FROM review WHERE tem_PRODUCT_id=1 ORDER BY helpful ALLOW FILTERING'

In [24]:
ans = session.execute(sql)

InvalidRequest: Error from server: code=2200 [Invalid query] message="ORDER BY is only supported when the partition key is restricted by an EQ or an IN."

In [18]:
for result in ans:
    print(result)

Row(helpful=9, rating=5, tem_product_id=1, customer='A2JW67OY8U6HHK', data=datetime.datetime(2000, 7, 28, 4, 0))
Row(helpful=5, rating=5, tem_product_id=1, customer='A2VE83MZF98ITY', data=datetime.datetime(2003, 12, 14, 4, 0))


(b) Dado um produto, listar os produtos similares com maiores vendas do que ele

(c) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

(d) Listar os 10 produtos lideres de venda em cada grupo de produtos

(e) Listar os 10 produtos com a maior média de avaliações úteis positivas por produto

(f) Listar a 5 categorias de produto com a maior média de avaliações úteis positivas por produto

In [38]:
sql = 'SELECT categorie, avg(helpful) FROM product_categorie, review WHERE product_categorie.product_categorie_id = review.tem_product_id GROUP BY categorie ORDER BY avg(helpful) DESC LIMIT 5'

In [39]:
session.execute(sql)

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:53 no viable alternative at input ',' (...categorie, avg(helpful) FROM [product_categorie],...)">

(g) Listar os 10 clientes que mais fizeram comentários por grupo de produto